In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image # For handling the images
import numpy as np
import pandas as pd


# Import different Keras functionalities
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Flatten, Dropout, concatenate
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf

from pathlib import Path

In [ ]:
path_subjects = "../DATA/dogs-vs-cats/train"
for f_name in os.listdir(path_subjects):
    img = Image.open(path_subjects + '/' + f_name)
    plt.imshow(img, cmap='rgb', vmin=0, vmax=255)
    plt.title('subject: ' + str(f_name))
    plt.show()

In [ ]:
weights_path = './FFNN-results/weights.h5'
loss_img_name = './FFNN-results/loss.png'
accuracy_img_name = './FFNN-results/accuracy.png'

weights_file = Path(weights_path)

In [3]:
# Parameters that characterizes the images, size and image type
img_width=128
img_height=128
img_channels=3
img_mode='rgb'

In [5]:
filenames=os.listdir("../DATA/dogs-vs-cats/train")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
def createNetwork:
    return model